# Monte Carlo and VaR

Value-at-risk(VaR) gives an indication of **how much you stand to lose on a portfolio with a given probability, over a specific time period.**

* VaR can be interpreted as *quantile value*: what values does the portfolio take on in the worst 5% of cases?
* VaR can also be interpreted of *confidence intervals*: we are $(1-\alpha)$-percent centain that the value of the portfolio will not fall below $VaR_\alpha(t, T)$
* VaR value is positive.

VaR models:
* Monte Carlo VaR Models
* Historical VaR Models
* Conditional Tail VaR

In [1]:
import numpy as np
from scipy.stats import norm
import numpy.matlib
from scipy.stats import uniform
import matplotlib.pyplot as plt
import math as m
import random as r

In [2]:
# General share info
S_0 = np.array([[100],[95],[50]])   # assets price
sigma = np.array([[0.15],[0.2],[0.3]]) # volatility
corr_mat = np.array([[1, 0.2, 0.4],[0.2, 1, 0.8],[0.4, 0.8, 1]])  # correlation Matrix
L = np.linalg.cholesky(corr_mat)  # Cholesky Decomposition
r = 0.1  # Risk-Free Rate
T = 1 # Time Horizon 1 year

In [3]:
np.random.seed(0)
t_simulations = 10000
alpha = 0.05
# current portfolio value
portval_current = np.sum(S_0)

In [4]:
def terminal_shareprice(S_0, risk_free_rate, sigma, Z, T):
    """
    Generates the terminal share price given some random normal values, z
    """
    # It returns an array of terminal stock prices.
    return S_0*np.exp((risk_free_rate-sigma**2/2)*T+sigma*np.sqrt(T)*Z)

In [5]:
# Creating 10000 simulations for future portfolio values
Z = np.matmul(L, norm.rvs(size= [3, t_simulations]))
portval_future = np.sum(terminal_shareprice(S_0, r, sigma, Z, T), axis=0)

In [7]:
# Calculating portfolio returns
port_return = (portval_future - portval_current)/portval_current

# Sorting the Returns
port_return = np.sort(port_return)

# Determining VaR
mVar_estimate = -port_return[int (np.floor(alpha*t_simulations))-1]
mVar_estimate

0.15694930298355253

## Historical data

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as pdr

In [17]:
# import data
def get_data(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start, end)
    print("==?", stockData)
    stockData = stockData['Close']
    
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

In [18]:
stockList = ['BNS.TO', 'GOOGL', 'XOM', 'NIO', 'KO', 'PEP','ARCC','IBM','AGNC','LCID']
stocks = [stock for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300)

meanReturns, covMatrix = get_data(stocks, startDate, endDate)

weights = np.random.random(len(meanReturns))
weights /= np.sum(weights)

TypeError: string indices must be integers, not 'str'

## Resources

* https://medium.com/the-quant-journey/monte-carlo-methods-for-risk-management-var-estimation-in-python-1f42d4b0d574
* https://medium.com/codex/measuring-portfolio-risk-using-monte-carlo-simulation-in-python-part-2-9297889588e8